# Gibbs free energy expressions for IN625 from ternary CALPHAD DB

In [1]:
import numpy as np
from pycalphad import Database, calculate, Model
from sympy.utilities.codegen import codegen
from sympy.parsing.sympy_parser import parse_expr
from sympy import diff, symbols

In [3]:
#tdb = Database('mc_ni_v2.033.tdb')
tdb = Database('Thermotech_Mo-Nb-Ni_FCC-delta-mu-BCC.tdb')

In [4]:
phases = ['LIQUID', 'FCC_A1', 'MU_PHASE', 'DELTA', 'BCC_A2']
elements = ['NI', 'MO', 'NB', 'VA']
temp = 870 + 273.15 # Kelvin

## Export Gibbs functions using SymPy

In [5]:
c_gamma = list(set([i for c in tdb.phases['FCC_A1'].constituents for i in c]))
m_gamma = Model(tdb, c_gamma, 'FCC_A1')

#FCC_A10NI, FCC_A11MO, FCC_A10NB = symbols('FCC_A10NI FCC_A11MO FCC_A10NB')

g_gamma = parse_expr(str(m_gamma.ast))
#print expr

#dm = diff(expr, MU_PHASE2NI)
#print dm

#dm2 = diff(expr, MU_PHASE2NI, MU_PHASE2NI)
#print dm2

#codegen([('energy',expr),('first_der',dm),('second_der',dm2)], language='C', to_files=True)

In [6]:
c_mu = list(set([i for c in tdb.phases['MU_PHASE'].constituents for i in c]))
m_mu = Model(tdb, c_mu, 'MU_PHASE')

#MU_PHASE1NI, MU_PHASE1MO, MU_PHASE1NB = symbols('MU_PHASE1NI MU_PHASE1MO MU_PHASE1NB')

g_mu = parse_expr(str(m_mu.ast))

In [7]:
c_delta = list(set([i for c in tdb.phases['DELTA'].constituents for i in c]))
m_delta = Model(tdb, c_delta, 'DELTA')

#MU_PHASE1NI, MU_PHASE1MO, MU_PHASE1NB = symbols('MU_PHASE1NI MU_PHASE1MO MU_PHASE1NB')

g_delta = parse_expr(str(m_delta.ast))

In [8]:
c_bcc = list(set([i for c in tdb.phases['BCC_A2'].constituents for i in c]))
m_bcc = Model(tdb, c_bcc, 'BCC_A2')

#MU_PHASE1NI, MU_PHASE1MO, MU_PHASE1NB = symbols('MU_PHASE1NI MU_PHASE1MO MU_PHASE1NB')

g_bcc = parse_expr(str(m_bcc.ast))

In [9]:
codegen([('g_gam',g_gamma), ('g_mu',g_mu), ('g_del',g_delta), ('g_bcc',g_bcc)],
        language='C', prefix='energy625', project='ALLOY625', to_files=True)

In [10]:
print m_gamma.site_ratios
print m_delta.site_ratios
print m_mu.site_ratios
print m_bcc.site_ratios

(1.0, 1.0)
(3.0, 1.0)
(6.0, 4.0, 3.0)
(1.0, 3.0)


## SymPy system of equations

In [ ]:
from sympy import Eq, solve
a1, a2, a3 = symbols('a1 a2 a3')
xMO, xNB, xNI = symbols('xMO xNB xNI')

In [29]:
# Delta phase: Cannot be solved without more information,
#           would need to iteratively determine internal
#           phase equilibrium in terms of {y}.
#           Unless one of the second sublattice's constituents
#           doesn't vary much in the temperature region of interest...

y1NB, y1NI, y2MO, y2NB, y2NI = symbols('y1NB y1NI y2MO y2NB y2NI')

ed1 = Eq(a1*y1NB + a2*y2NB, xNB)
ed2 = Eq(a1*y1NI + a2*y2NI, xNI)
ed3 = Eq(a2*y2MO, xMO)
ed4 = Eq(y1NB + y1NI, 1)
ed5 = Eq(y2MO + y2NB + y2NI, 1)

solve([ed1,ed2,ed3,ed4,ed5],[y1NB,y1NI,y2MO,y2NB,y2NI])

[]

In [31]:
# mu phase: Cannot be solved without more information,
#           would need to iteratively determine internal
#           phase equilibrium in terms of {y}.
#           Unless one of the second sublattice's constituents
#           doesn't vary much in the temperature region of interest...

y1NI, y2MO, y2NB, y3MO, y3NB, y3NI, Z = symbols('y1NI y2MO y2NB y3MO y3NB y3NI Z')

em1 = Eq(a1*y1NI + a3*y3NI, xNI)
em2 = Eq(a2*y2MO + a3*y3MO, xMO)
em3 = Eq(a2*y2NB + a3*y3NB, xNB)
em4 = Eq(y1NI, 1)
em5 = Eq(y2MO + y2NB, 1)
em6 = Eq(y3MO + y3NB + y3NI, 1)
em7 = Eq(y2NB,Z)

solve([em1,em2,em3,em4,em5,em6],[y1NI,y2MO,y2NB,y3MO,y3NB,y3NI])

[]